In [42]:
#!pip install -U aeon
#!pip install aeon[all_extras]
import warnings
import pandas as pd
from matplotlib import pyplot as plt
from aeon.visualisation import plot_series
from sklearn.metrics import mean_absolute_percentage_error as mape
from all_functions import *
import os
from aeon.clustering.averaging import elastic_barycenter_average
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
from VotingCombination import VotingCombination
from tslearn.barycenters import \
    euclidean_barycenter, \
    dtw_barycenter_averaging, \
    dtw_barycenter_averaging_subgradient, \
    softdtw_barycenter
warnings.filterwarnings("ignore")
%matplotlib inline

def get_predictions_csv(path, format, start_index):
    df = pd.read_csv(path, sep=";")
    results = {}
    params = {}
    # for format in formats:
    filtered_df = df[df['DATA'] == format]
    
    columns_p1_to_p12 = filtered_df.loc[:, 'P1':'P12']
    
    # values_list = columns_p1_to_p12.values.tolist()
    values_list = columns_p1_to_p12.values.flatten().tolist()     
    results = pd.Series(values_list, index=start_index)
    params = filtered_df['PARAMS'].iloc[0]
            
    return results, params

def get_params_model(caminho_arquivo, transformation):
    df = pd.read_csv(caminho_arquivo, sep=';')
    
    df_filtrado = df[df['DATA'] == transformation]
    params_dict = ast.literal_eval(df_filtrado['PARAMS'].iloc[0])
    
    return params_dict

In [43]:
derivado = "etanolhidratado"
estado = "SP"
horizon = 12
window = 12
series = read_series(f'../datasets/venda/mensal/uf/{derivado}/mensal_{estado.lower()}_{derivado}.csv')
data = rolling_window(series, window)
data = data.dropna()
X_train, X_test, y_train, _ = train_test_split(data, horizon)
train2, test2 = train_test_stats(series, horizon)

In [44]:
train_test_splits = []
initial_train_size = 24
test_size = 12
start = 0

current_train_size = initial_train_size
while start + current_train_size + test_size <= len(series):
    end_train = start + current_train_size
    end_test = end_train + test_size
    
    train_data = series.iloc[start:end_train]
    test_data = series.iloc[end_train:end_test]
    
    train_test_splits.append((train_data, test_data))
    
    current_train_size += test_size 


In [61]:
params = get_params_model(f'./paper_final/arima/{derivado}/transform_{estado}.csv', 'normal')
i = (params['p'], params['d'], params['q'])
i

(18, 2, 22)

In [47]:
# models= ['xgb', 'rf', 'knn']
# transformations = ['normal', 'log']
# model_paths = [f'./paper2/{model}' for model in models]
# # results_combination = f"./paper/{'_'.join(models)}"
# results_combination = "./paper2/fusion_barycenter_regressors"
# horizon = 12
# window = 12

# colunas = ['DATA', 'MCPM', 'UF', 'PRODUCT', 'MODEL', 'PARAMS', 'WINDOW', 'HORIZON', 'RMSE', 'MAPE', 'POCID', 'PBE', 'MASE',
#            'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12',
#            ]
# for subdir, _, files in os.walk(model_paths[0]):
#     for file in files:
#         if file.endswith('.csv'):
#             uf = file.split("_")[1][:2]
#             derivado = subdir.split("/")[-1]
#             # file_path = os.path.join(subdir, file)
#             series = read_series(f"../datasets/venda/mensal/uf/{derivado}/mensal_{uf.lower()}_{derivado}.csv")
#             _, test = train_test_stats(series, horizon)
#             predictions = {}

#             path_derivado = results_combination + f"/{derivado}"
#             preds_tf = []
#             preds_elastic = []
#             for path in model_paths:
#                 for tf in transformations:
#                     model = path.split('/')[2]
#                     preds, params = get_predictions_csv(f"{path}/{derivado}/transform_{uf}.csv", tf, start_index=test.index)
#                     preds_tf.append((preds, model, tf))
#                     preds_elastic.append(np.array([[preds.values]]))

#             y_baseline = series[-horizon*2:-horizon].values

#             for p, model, tf in preds_tf:
#                 predictions[f'{model}_{tf}'] = pd.Series(p, index=test.index)
                
#             barycenters = [preds for preds, _ , _ in preds_tf]
#             print(barycenters)


#             #MAX
#             voting = VotingCombination(predictions, combination='max')
#             preds_max = voting.predict()
#             media = np.mean(preds_max)
#             desvio = np.std(preds_max)

            
#             rmse_result = rmse(test, preds_max)
#             mape_result = mape(test, preds_max)
#             pocid_result = pocid(test, preds_max)
#             pbe_result = pbe(test, preds_max)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_max, y_baseline)

#             path_derivado = results_combination + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             os.makedirs(path_derivado, exist_ok=True)

#             if not os.path.exists(csv_file):
#                 pd.DataFrame(columns=colunas).to_csv(csv_file, sep=';', index=False)

#             df_result = pd.DataFrame({'DATA': "max", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL': f"{'_'.join(models)}", 'PARAMS':  str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_max[0], 'P2': preds_max[1], 'P3': preds_max[2], 'P4': preds_max[3], 'P5': preds_max[4],
#                                             'P6': preds_max[5], 'P7': preds_max[6], 'P8': preds_max[7], 'P9': preds_max[8], 'P10': preds_max[9],
#                                             'P11': preds_max[10], 'P12': preds_max[11]
#                                             }, index=[0])
#             df_result = df_result[colunas]

#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #elastic barycenter
#             capt = np.vstack(preds_elastic)
#             elastic = elastic_barycenter_average(capt, distance="twe", reach=15)
#             preds_elastic = pd.Series(elastic[0].tolist(), index=test.index)
#             preds_elastic = znorm(preds_elastic)
#             preds_elastic = znorm_reverse(preds_elastic, media, desvio)

#             rmse_result = rmse(test, preds_elastic)
#             mape_result = mape(test, preds_elastic)
#             pocid_result = pocid(test, preds_elastic)
#             pbe_result = pbe(test, preds_elastic)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_elastic, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             os.makedirs(path_derivado, exist_ok=True)

#             if not os.path.exists(csv_file):
#                 pd.DataFrame(columns=colunas).to_csv(csv_file, sep=';', index=False)
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "twe", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_elastic[0], 'P2': preds_elastic[1], 'P3': preds_elastic[2], 'P4': preds_elastic[3], 'P5': preds_elastic[4],
#                                             'P6': preds_elastic[5], 'P7': preds_elastic[6], 'P8': preds_elastic[7], 'P9': preds_elastic[8], 'P10': preds_elastic[9],
#                                             'P11': preds_elastic[10], 'P12': preds_elastic[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)

#             voting = VotingCombination(predictions, combination='min')
#             preds_mean = voting.predict()
#             preds_mean = znorm(preds_mean)
#             preds_mean = znorm_reverse(preds_mean, media, desvio)
#             rmse_result = rmse(test, preds_mean)
#             mape_result = mape(test, preds_mean)
#             pocid_result = pocid(test, preds_mean)
#             pbe_result = pbe(test, preds_mean)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_mean, y_baseline)

#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             os.makedirs(path_derivado, exist_ok=True)

#             if not os.path.exists(csv_file):
#                 pd.DataFrame(columns=colunas).to_csv(csv_file, sep=';', index=False)

#             df_result = pd.DataFrame({'DATA': "min", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL': f"{'_'.join(models)}", 'PARAMS':  str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_mean[0], 'P2': preds_mean[1], 'P3': preds_mean[2], 'P4': preds_mean[3], 'P5': preds_mean[4],
#                                             'P6': preds_mean[5], 'P7': preds_mean[6], 'P8': preds_mean[7], 'P9': preds_mean[8], 'P10': preds_mean[9],
#                                             'P11': preds_mean[10], 'P12': preds_mean[11]
#                                             }, index=[0])
#             df_result = df_result[colunas]

#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             # #MEDIAN
#             voting = VotingCombination(predictions, combination='median')
#             preds_median = voting.predict()
#             preds_median = znorm(preds_median)
#             preds_median = znorm_reverse(preds_median, media, desvio)

#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_median)
#             mape_result = mape(test, preds_median)
#             pocid_result = pocid(test, preds_median)
#             pbe_result = pbe(test, preds_median)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_median, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "median", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_median[0], 'P2': preds_median[1], 'P3': preds_median[2], 'P4': preds_median[3], 'P5': preds_median[4],
#                                             'P6': preds_median[5], 'P7': preds_median[6], 'P8': preds_median[7], 'P9': preds_median[8], 'P10': preds_median[9],
#                                             'P11': preds_median[10], 'P12': preds_median[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #softdtw_barycenter
#             preds_softdtw = softdtw_barycenter(barycenters, max_iter=35)
#             preds_softdtw = pd.Series(preds_softdtw.flatten().tolist(), index=test.index)
#             preds_softdtw = znorm(preds_softdtw)
#             preds_softdtw = znorm_reverse(preds_softdtw, media, desvio)

#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_softdtw)
#             mape_result = mape(test, preds_softdtw)
#             pocid_result = pocid(test, preds_softdtw)
#             pbe_result = pbe(test, preds_softdtw)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_softdtw, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "softdtw", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_softdtw[0], 'P2': preds_softdtw[1], 'P3': preds_softdtw[2], 'P4': preds_softdtw[3], 'P5': preds_softdtw[4],
#                                             'P6': preds_softdtw[5], 'P7': preds_softdtw[6], 'P8': preds_softdtw[7], 'P9': preds_softdtw[8], 'P10': preds_softdtw[9],
#                                             'P11': preds_softdtw[10], 'P12': preds_softdtw[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             preds_softdtw = softdtw_barycenter(barycenters, max_iter=35, gamma=0.01)
#             preds_softdtw = pd.Series(preds_softdtw.flatten().tolist(), index=test.index)
#             preds_softdtw = znorm(preds_softdtw)
#             preds_softdtw = znorm_reverse(preds_softdtw, media, desvio)

#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_softdtw)
#             mape_result = mape(test, preds_softdtw)
#             pocid_result = pocid(test, preds_softdtw)
#             pbe_result = pbe(test, preds_softdtw)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_softdtw, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "softdtw0.01", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_softdtw[0], 'P2': preds_softdtw[1], 'P3': preds_softdtw[2], 'P4': preds_softdtw[3], 'P5': preds_softdtw[4],
#                                             'P6': preds_softdtw[5], 'P7': preds_softdtw[6], 'P8': preds_softdtw[7], 'P9': preds_softdtw[8], 'P10': preds_softdtw[9],
#                                             'P11': preds_softdtw[10], 'P12': preds_softdtw[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)



#             preds_softdtw = softdtw_barycenter(barycenters, max_iter=35, gamma=0.1)
#             preds_softdtw = pd.Series(preds_softdtw.flatten().tolist(), index=test.index)
#             preds_softdtw = znorm(preds_softdtw)
#             preds_softdtw = znorm_reverse(preds_softdtw, media, desvio)

#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_softdtw)
#             mape_result = mape(test, preds_softdtw)
#             pocid_result = pocid(test, preds_softdtw)
#             pbe_result = pbe(test, preds_softdtw)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_softdtw, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "softdtw0.1", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_softdtw[0], 'P2': preds_softdtw[1], 'P3': preds_softdtw[2], 'P4': preds_softdtw[3], 'P5': preds_softdtw[4],
#                                             'P6': preds_softdtw[5], 'P7': preds_softdtw[6], 'P8': preds_softdtw[7], 'P9': preds_softdtw[8], 'P10': preds_softdtw[9],
#                                             'P11': preds_softdtw[10], 'P12': preds_softdtw[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #dtw_barycenter_averaging_subgradient
#             preds_dtw_subgradient = dtw_barycenter_averaging_subgradient(barycenters, max_iter=35)
#             preds_dtw_subgradient = pd.Series(preds_dtw_subgradient.flatten().tolist(), index=test.index)
#             preds_dtw_subgradient = znorm(preds_dtw_subgradient)
#             preds_dtw_subgradient = znorm_reverse(preds_dtw_subgradient, media, desvio)

#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_dtw_subgradient)
#             mape_result = mape(test, preds_dtw_subgradient)
#             pocid_result = pocid(test, preds_dtw_subgradient)
#             pbe_result = pbe(test, preds_dtw_subgradient)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_dtw_subgradient, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "dtw_avg_subgradient", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_dtw_subgradient[0], 'P2': preds_dtw_subgradient[1], 'P3': preds_dtw_subgradient[2], 'P4': preds_dtw_subgradient[3], 'P5': preds_dtw_subgradient[4],
#                                             'P6': preds_dtw_subgradient[5], 'P7': preds_dtw_subgradient[6], 'P8': preds_dtw_subgradient[7], 'P9': preds_dtw_subgradient[8], 'P10': preds_dtw_subgradient[9],
#                                             'P11': preds_dtw_subgradient[10], 'P12': preds_dtw_subgradient[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #dtw_barycenter_averaging
#             preds_dtw_avg = dtw_barycenter_averaging(barycenters, max_iter=35)
#             preds_dtw_avg = pd.Series(preds_dtw_avg.flatten().tolist(), index=test.index)
#             preds_dtw_avg = znorm(preds_dtw_avg)
#             preds_dtw_avg = znorm_reverse(preds_dtw_avg, media, desvio)

#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_dtw_avg)
#             mape_result = mape(test, preds_dtw_avg)
#             pocid_result = pocid(test, preds_dtw_avg)
#             pbe_result = pbe(test, preds_dtw_avg)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_dtw_avg, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "dtw_avg", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_dtw_avg[0], 'P2': preds_dtw_avg[1], 'P3': preds_dtw_avg[2], 'P4': preds_dtw_avg[3], 'P5': preds_dtw_avg[4],
#                                             'P6': preds_dtw_avg[5], 'P7': preds_dtw_avg[6], 'P8': preds_dtw_avg[7], 'P9': preds_dtw_avg[8], 'P10': preds_dtw_avg[9],
#                                             'P11': preds_dtw_avg[10], 'P12': preds_dtw_avg[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #euclidean_barycenter
#             preds_euclidean = euclidean_barycenter(barycenters)
#             preds_euclidean = pd.Series(preds_euclidean.flatten().tolist(), index=test.index)
#             preds_euclidean = znorm(preds_euclidean)
#             preds_euclidean = znorm_reverse(preds_euclidean, media, desvio)

#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_euclidean)
#             mape_result = mape(test, preds_euclidean)
#             pocid_result = pocid(test, preds_euclidean)
#             pbe_result = pbe(test, preds_euclidean)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_euclidean, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "euclidean", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_euclidean[0], 'P2': preds_euclidean[1], 'P3': preds_euclidean[2], 'P4': preds_euclidean[3], 'P5': preds_euclidean[4],
#                                             'P6': preds_euclidean[5], 'P7': preds_euclidean[6], 'P8': preds_euclidean[7], 'P9': preds_euclidean[8], 'P10': preds_euclidean[9],
#                                             'P11': preds_euclidean[10], 'P12': preds_euclidean[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)
            

In [48]:
# models= ['xgb', 'rf', 'knn']
# transformations = ['normal', 'log']
# model_paths = [f'./paper2/{model}' for model in models]
# # results_combination = f"./paper/{'_'.join(models)}"
# results_combination = "./paper2/simple_combination_regressors"
# horizon = 12
# window = 12

# colunas = ['DATA', 'MCPM', 'UF', 'PRODUCT', 'MODEL', 'PARAMS', 'WINDOW', 'HORIZON', 'RMSE', 'MAPE', 'POCID', 'PBE', 'MASE',
#            'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12',
#            ]
# for subdir, _, files in os.walk(model_paths[0]):
#     for file in files:
#         if file.endswith('.csv'):
#             uf = file.split("_")[1][:2]
#             derivado = subdir.split("/")[-1]
#             # file_path = os.path.join(subdir, file)
#             series = read_series(f"../datasets/venda/mensal/uf/{derivado}/mensal_{uf.lower()}_{derivado}.csv")
#             _, test = train_test_stats(series, horizon)
#             predictions = {}

#             path_derivado = results_combination + f"/{derivado}"
#             preds_tf = []
#             preds_elastic = []
#             for path in model_paths:
#                 for tf in transformations:
#                     model = path.split('/')[2]
#                     preds, params = get_predictions_csv(f"{path}/{derivado}/transform_{uf}.csv", tf, start_index=test.index)
#                     preds_tf.append((preds, model, tf))
#                     preds_elastic.append(np.array([[preds.values]]))

#             y_baseline = series[-horizon*2:-horizon].values

#             for p, model, tf in preds_tf:
#                 predictions[f'{model}_{tf}'] = pd.Series(p, index=test.index)
                
#             barycenters = [preds for preds, _ , _ in preds_tf]

#             #elastic barycenter
#             capt = np.vstack(preds_elastic)
#             elastic = elastic_barycenter_average(capt, distance="twe", reach=15)
#             preds_elastic = pd.Series(elastic[0].tolist(), index=test.index)


#             rmse_result = rmse(test, preds_elastic)
#             mape_result = mape(test, preds_elastic)
#             pocid_result = pocid(test, preds_elastic)
#             pbe_result = pbe(test, preds_elastic)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_elastic, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             os.makedirs(path_derivado, exist_ok=True)

#             if not os.path.exists(csv_file):
#                 pd.DataFrame(columns=colunas).to_csv(csv_file, sep=';', index=False)
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "twe", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_elastic[0], 'P2': preds_elastic[1], 'P3': preds_elastic[2], 'P4': preds_elastic[3], 'P5': preds_elastic[4],
#                                             'P6': preds_elastic[5], 'P7': preds_elastic[6], 'P8': preds_elastic[7], 'P9': preds_elastic[8], 'P10': preds_elastic[9],
#                                             'P11': preds_elastic[10], 'P12': preds_elastic[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)




#             voting = VotingCombination(predictions, combination='min')
#             preds_mean = voting.predict()
#             rmse_result = rmse(test, preds_mean)
#             mape_result = mape(test, preds_mean)
#             pocid_result = pocid(test, preds_mean)
#             pbe_result = pbe(test, preds_mean)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_mean, y_baseline)

#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             os.makedirs(path_derivado, exist_ok=True)

#             if not os.path.exists(csv_file):
#                 pd.DataFrame(columns=colunas).to_csv(csv_file, sep=';', index=False)

#             df_result = pd.DataFrame({'DATA': "min", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL': f"{'_'.join(models)}", 'PARAMS':  str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_mean[0], 'P2': preds_mean[1], 'P3': preds_mean[2], 'P4': preds_mean[3], 'P5': preds_mean[4],
#                                             'P6': preds_mean[5], 'P7': preds_mean[6], 'P8': preds_mean[7], 'P9': preds_mean[8], 'P10': preds_mean[9],
#                                             'P11': preds_mean[10], 'P12': preds_mean[11]
#                                             }, index=[0])
#             df_result = df_result[colunas]

#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             voting = VotingCombination(predictions, combination='max')
#             preds_mean = voting.predict()
#             rmse_result = rmse(test, preds_mean)
#             mape_result = mape(test, preds_mean)
#             pocid_result = pocid(test, preds_mean)
#             pbe_result = pbe(test, preds_mean)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_mean, y_baseline)

#             path_derivado = results_combination + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             os.makedirs(path_derivado, exist_ok=True)

#             if not os.path.exists(csv_file):
#                 pd.DataFrame(columns=colunas).to_csv(csv_file, sep=';', index=False)

#             df_result = pd.DataFrame({'DATA': "max", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL': f"{'_'.join(models)}", 'PARAMS':  str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_mean[0], 'P2': preds_mean[1], 'P3': preds_mean[2], 'P4': preds_mean[3], 'P5': preds_mean[4],
#                                             'P6': preds_mean[5], 'P7': preds_mean[6], 'P8': preds_mean[7], 'P9': preds_mean[8], 'P10': preds_mean[9],
#                                             'P11': preds_mean[10], 'P12': preds_mean[11]
#                                             }, index=[0])
#             df_result = df_result[colunas]

#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)

#             # #MEDIAN
#             voting = VotingCombination(predictions, combination='median')
#             preds_median = voting.predict()

#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_median)
#             mape_result = mape(test, preds_median)
#             pocid_result = pocid(test, preds_median)
#             pbe_result = pbe(test, preds_median)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_median, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "median", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_median[0], 'P2': preds_median[1], 'P3': preds_median[2], 'P4': preds_median[3], 'P5': preds_median[4],
#                                             'P6': preds_median[5], 'P7': preds_median[6], 'P8': preds_median[7], 'P9': preds_median[8], 'P10': preds_median[9],
#                                             'P11': preds_median[10], 'P12': preds_median[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #softdtw_barycenter
#             preds_softdtw = softdtw_barycenter(barycenters, max_iter=35)
#             preds_softdtw = pd.Series(preds_softdtw.flatten().tolist(), index=test.index)


#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_softdtw)
#             mape_result = mape(test, preds_softdtw)
#             pocid_result = pocid(test, preds_softdtw)
#             pbe_result = pbe(test, preds_softdtw)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_softdtw, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "softdtw", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_softdtw[0], 'P2': preds_softdtw[1], 'P3': preds_softdtw[2], 'P4': preds_softdtw[3], 'P5': preds_softdtw[4],
#                                             'P6': preds_softdtw[5], 'P7': preds_softdtw[6], 'P8': preds_softdtw[7], 'P9': preds_softdtw[8], 'P10': preds_softdtw[9],
#                                             'P11': preds_softdtw[10], 'P12': preds_softdtw[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             preds_softdtw = softdtw_barycenter(barycenters, max_iter=35, gamma=0.01)
#             preds_softdtw = pd.Series(preds_softdtw.flatten().tolist(), index=test.index)


#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_softdtw)
#             mape_result = mape(test, preds_softdtw)
#             pocid_result = pocid(test, preds_softdtw)
#             pbe_result = pbe(test, preds_softdtw)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_softdtw, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "softdtw0.01", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_softdtw[0], 'P2': preds_softdtw[1], 'P3': preds_softdtw[2], 'P4': preds_softdtw[3], 'P5': preds_softdtw[4],
#                                             'P6': preds_softdtw[5], 'P7': preds_softdtw[6], 'P8': preds_softdtw[7], 'P9': preds_softdtw[8], 'P10': preds_softdtw[9],
#                                             'P11': preds_softdtw[10], 'P12': preds_softdtw[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)



#             preds_softdtw = softdtw_barycenter(barycenters, max_iter=35, gamma=0.1)
#             preds_softdtw = pd.Series(preds_softdtw.flatten().tolist(), index=test.index)


#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_softdtw)
#             mape_result = mape(test, preds_softdtw)
#             pocid_result = pocid(test, preds_softdtw)
#             pbe_result = pbe(test, preds_softdtw)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_softdtw, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "softdtw0.1", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_softdtw[0], 'P2': preds_softdtw[1], 'P3': preds_softdtw[2], 'P4': preds_softdtw[3], 'P5': preds_softdtw[4],
#                                             'P6': preds_softdtw[5], 'P7': preds_softdtw[6], 'P8': preds_softdtw[7], 'P9': preds_softdtw[8], 'P10': preds_softdtw[9],
#                                             'P11': preds_softdtw[10], 'P12': preds_softdtw[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #dtw_barycenter_averaging_subgradient
#             preds_dtw_subgradient = dtw_barycenter_averaging_subgradient(barycenters, max_iter=35)
#             preds_dtw_subgradient = pd.Series(preds_dtw_subgradient.flatten().tolist(), index=test.index)


#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_dtw_subgradient)
#             mape_result = mape(test, preds_dtw_subgradient)
#             pocid_result = pocid(test, preds_dtw_subgradient)
#             pbe_result = pbe(test, preds_dtw_subgradient)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_dtw_subgradient, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "dtw_avg_subgradient", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_dtw_subgradient[0], 'P2': preds_dtw_subgradient[1], 'P3': preds_dtw_subgradient[2], 'P4': preds_dtw_subgradient[3], 'P5': preds_dtw_subgradient[4],
#                                             'P6': preds_dtw_subgradient[5], 'P7': preds_dtw_subgradient[6], 'P8': preds_dtw_subgradient[7], 'P9': preds_dtw_subgradient[8], 'P10': preds_dtw_subgradient[9],
#                                             'P11': preds_dtw_subgradient[10], 'P12': preds_dtw_subgradient[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #dtw_barycenter_averaging
#             preds_dtw_avg = dtw_barycenter_averaging(barycenters, max_iter=35)
#             preds_dtw_avg = pd.Series(preds_dtw_avg.flatten().tolist(), index=test.index)


#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_dtw_avg)
#             mape_result = mape(test, preds_dtw_avg)
#             pocid_result = pocid(test, preds_dtw_avg)
#             pbe_result = pbe(test, preds_dtw_avg)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_dtw_avg, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "dtw_avg", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_dtw_avg[0], 'P2': preds_dtw_avg[1], 'P3': preds_dtw_avg[2], 'P4': preds_dtw_avg[3], 'P5': preds_dtw_avg[4],
#                                             'P6': preds_dtw_avg[5], 'P7': preds_dtw_avg[6], 'P8': preds_dtw_avg[7], 'P9': preds_dtw_avg[8], 'P10': preds_dtw_avg[9],
#                                             'P11': preds_dtw_avg[10], 'P12': preds_dtw_avg[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)


#             #euclidean_barycenter
#             preds_euclidean = euclidean_barycenter(barycenters)
#             preds_euclidean = pd.Series(preds_euclidean.flatten().tolist(), index=test.index)


#             y_baseline = series[-horizon*2:-horizon].values
#             rmse_result = rmse(test, preds_euclidean)
#             mape_result = mape(test, preds_euclidean)
#             pocid_result = pocid(test, preds_euclidean)
#             pbe_result = pbe(test, preds_euclidean)
#             mcpm_result = mcpm(rmse_result, mape_result, pocid_result)
#             mase_result = mase(test, preds_euclidean, y_baseline)

#             # path_derivado = results_median + f"/{derivado}"
#             csv_file = path_derivado + f"/transform_{uf.upper()}.csv"
#             # os.makedirs(path_derivado, exist_ok=True)
#             df_result = pd.DataFrame({'DATA': "euclidean", 'MCPM': mcpm_result, 'UF': uf, 'PRODUCT': derivado, 'MODEL':  f"{'_'.join(models)}", 'PARAMS': str({}), 'WINDOW': window, 'HORIZON': horizon,  
#                                             'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result,
#                                             'P1': preds_euclidean[0], 'P2': preds_euclidean[1], 'P3': preds_euclidean[2], 'P4': preds_euclidean[3], 'P5': preds_euclidean[4],
#                                             'P6': preds_euclidean[5], 'P7': preds_euclidean[6], 'P8': preds_euclidean[7], 'P9': preds_euclidean[8], 'P10': preds_euclidean[9],
#                                             'P11': preds_euclidean[10], 'P12': preds_euclidean[11]
#                                             }, index=[0])
#             # df_result = df_result[colunas]
#             df_result.to_csv(csv_file, sep=';', mode='a', header=False, index=False)
            